In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import autokeras as ak

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

### importation de  la donnée de prédictions

In [2]:
test_data = pd.read_csv('/home/apprenant/PycharmProjects/Brief-10-Employees-Turnover/data/attrition_test.csv')
train_data = pd.read_csv('/home/apprenant/PycharmProjects/Brief-10-Employees-Turnover/data/attrition_train.csv')

In [3]:
x_test = test_data.drop(['EmployeeCount','index1','EmployeeNumber','Over18','StandardHours'], axis =1)
x_train = train_data.drop(['EmployeeCount','Attrition','EmployeeNumber','Over18','StandardHours'], axis =1)

In [4]:
num = x_train.select_dtypes(include = ["float64","int64"])
features_num = num.columns
print(features_num)

Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')


In [5]:
cat = x_train.select_dtypes(include = ["object"])
features_cat = cat.columns
print(features_cat)

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'OverTime'],
      dtype='object')


In [6]:
transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"), # there are a few missing values
    StandardScaler(),
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown='ignore'),
)


In [7]:
preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)

In [8]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [9]:

y_train = train_data['Attrition'].to_numpy().squeeze()

In [10]:
from pytorch_tabnet.tab_model import TabNetClassifier

classifier = TabNetClassifier(verbose=0,seed=42)
classifier.fit(X_train=x_train, y_train=y_train,
               patience=5,max_epochs=100,
               eval_metric=['auc'])

predictions = classifier.predict_proba(x_test)[:,1]

No early stopping will be performed, last training weights will be used.


# prediction

In [11]:
nb_zero = 0
nb_un = 0
for i in range(len(predictions)):
    if predictions[i] < 0.5:
        predictions[i]=0
        nb_zero +=1
    else:
        predictions[i]=1
        nb_un +=1

In [12]:
print("nb de zero :",nb_zero)
print("nb de un :",nb_un)

nb de zero : 343
nb de un : 27
